In [ ]:
import { Picker } from '@react-native-picker/picker';
import axios from 'axios';
import React, { useEffect, useState } from 'react';
import { Button, Dimensions, ScrollView, StyleSheet, Text } from 'react-native';
import { LineChart } from 'react-native-chart-kit';

const screenWidth = Dimensions.get('window').width;

export default function Dashboard() {
  const [data, setData] = useState([]);
  const [duration, setDuration] = useState(20);
  const [loading, setLoading] = useState(false);

  const fetchData = async () => {
    try {
      const res = await axios.get('https://api.thingspeak.com/channels/FN233C0UDKZWLUS9/feeds.json?results=20');
      setData(res.data.feeds);
    } catch (err) {
      console.log('Error fetching data:', err);
    }
  };

  const triggerManualWatering = async () => {
    try {
      setLoading(true);
      const message = `RUN:${duration}`;
      const response = await axios.post('a1eef8be216949238865abfec7ed13a2.s1.eu.hivemq.cloud', {
        topic: 'watering/manual',
        message,
      });
      console.log('MQTT message sent:', response.data);
    } catch (err) {
      console.error('Failed to send MQTT:', err);
    } finally {
      setLoading(false);
    }
  };

  useEffect(() => {
    fetchData();
  }, []);

  const extractChartData = (field: string) => {
    return {
      labels: data.map((_, idx) => idx.toString()),
      datasets: [
        {
          data: data.map(entry => parseFloat(entry[field]) || 0),
        },
      ],
    };
  };

  return (
    <ScrollView style={styles.container}>
      <Text style={styles.title}>Watering System Dashboard</Text>

      <Text style={styles.sectionTitle}>Manual Watering Duration (seconds)</Text>
      <Picker
        selectedValue={duration}
        onValueChange={(value) => setDuration(value)}
        style={styles.picker}
      >
        {[10, 20, 30, 40, 50, 60].map((val) => (
          <Picker.Item label={`${val}s`} value={val} key={val} />
        ))}
      </Picker>

      <Button
        title={loading ? 'Sending...' : 'Trigger Manual Watering'}
        onPress={triggerManualWatering}
        disabled={loading}
      />

      <Text style={styles.sectionTitle}>Temperature</Text>
      <LineChart
        data={extractChartData('field2')}
        width={screenWidth - 32}
        height={220}
        chartConfig={chartConfig}
      />

      <Text style={styles.sectionTitle}>Humidity</Text>
      <LineChart
        data={extractChartData('field3')}
        width={screenWidth - 32}
        height={220}
        chartConfig={chartConfig}
      />

      <Text style={styles.sectionTitle}>Soil Moisture</Text>
      <LineChart
        data={extractChartData('field1')}
        width={screenWidth - 32}
        height={220}
        chartConfig={chartConfig}
      />
    </ScrollView>
  );
}

const chartConfig = {
  backgroundGradientFrom: '#fff',
  backgroundGradientTo: '#fff',
  color: (opacity = 1) => `rgba(0, 122, 255, ${opacity})`,
  strokeWidth: 2,
};

const styles = StyleSheet.create({
  container: {
    padding: 16,
    backgroundColor: '#fff',
    flex: 1,
  },
  title: {
    fontSize: 24,
    fontWeight: 'bold',
    marginBottom: 16,
  },
  sectionTitle: {
    fontSize: 18,
    marginTop: 24,
    marginBottom: 8,
  },
  picker: {
    height: 50,
    width: '100%',
    marginBottom: 16,
  },
});